# iQuHack 2026 - Quantum Entanglement Distillation Game

In this game, you design quantum circuits to distill noisy Bell pairs and claim edges in a network.

**Game Flow**: Register -> Select Starting Node -> Design Circuits -> Claim Edges -> Score Points

In [ ]:
from client import GameClient
from visualization import GraphTool
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import json
from pathlib import Path

## Session Management

Save/load your session to avoid re-registering.

In [ ]:
SESSION_FILE = Path("session.json")

def save_session(client):
    if client.api_token:
        with open(SESSION_FILE, "w") as f:
            json.dump({"api_token": client.api_token, "player_id": client.player_id, "name": client.name}, f)
        print(f"Session saved.")

def load_session():
    if not SESSION_FILE.exists():
        return None
    with open(SESSION_FILE) as f:
        data = json.load(f)
    client = GameClient(api_token=data.get("api_token"))
    client.player_id = data.get("player_id")
    client.name = data.get("name")
    status = client.get_status()
    if status:
        print(f"Resumed: {client.player_id} | Score: {status.get('score', 0)} | Budget: {status.get('budget', 0)}")
        return client
    return None

In [ ]:
# Try to resume existing session
client = load_session()

if not client:
    print("No saved session. Register below.")

## Step 1: Register

Skip this if you resumed a session above.

In [ ]:
if client and client.api_token:
    print(f"Already registered as {client.player_id}")
else:
    client = GameClient()
    
    # CHANGE THESE to your unique values
    PLAYER_ID = "SleeplessShade"
    PLAYER_NAME = "Mogrel"
    
    result = client.register(PLAYER_ID, PLAYER_NAME, location=input("remote or in_person: ").strip())
    
    if result.get("ok"):
        print(f"Registered! Token: {client.api_token[:20]}...")
        candidates = result["data"].get("starting_candidates", [])
        print(f"\nStarting candidates ({len(candidates)}):")
        for c in candidates:
            print(f"  - {c['node_id']}: {c['utility_qubits']} qubits, +{c['bonus_bell_pairs']} bonus")
        save_session(client)
    else:
        print(f"Failed: {result.get('error', {}).get('message')}")

## Step 2: Select Starting Node

In [ ]:
status = client.get_status()

if status.get('starting_node'):
    print(f"Starting node: {status['starting_node']}")
    print(f"Budget: {status['budget']} | Score: {status['score']}")
else:
    print("Select a starting node from the candidates shown above.")
    # Uncomment and modify:
    result = client.select_starting_node("Singapore")
    print(result)

## Step 3: Explore the Network

In [ ]:
client.print_status()

In [ ]:
# Visualize the network (focused around your nodes)
viz = GraphTool(client.get_cached_graph())
owned = set(client.get_status().get('owned_nodes', []))
viz.render(owned)

## Step 4: Design a Distillation Circuit

For N Bell pairs, qubits are paired outside-in:
- Pair 0: qubits {0, 2N-1}
- Pair 1: qubits {1, 2N-2}
- ...
- Final pair: qubits {N-1, N}

Design a circuit that improves fidelity through LOCC operations.

In [ ]:
import numpy as np

def create_distillation_circuit(num_rounds: int = 1, use_dejmps: bool = False):
    """
    Entanglement distillation circuit supporting BBPSSW and DEJMPS variants.
    
    Args:
        num_rounds: Number of purification rounds (1-3). Uses N = num_rounds + 1 pairs.
        use_dejmps: If True, applies Rx(pi/2) twirling (DEJMPS style).
    
    Topology (N pairs, 2N qubits):
        Alice: qubits 0 to N-1
        Bob:   qubits N to 2N-1
        Pair i: (i, 2N-1-i)  [folded/nested]
        Survivor: pair N-1 = (N-1, N)
    """
    num_pairs = num_rounds + 1
    total_qubits = 2 * num_pairs
    N = num_pairs
    
    qr = QuantumRegister(total_qubits, 'q')
    mA = ClassicalRegister(num_rounds, 'mA')
    parity = ClassicalRegister(num_rounds, 'par')
    flag = ClassicalRegister(1, 'flag')
    qc = QuantumCircuit(qr, mA, parity, flag)
    
    def get_pair(i):
        """Returns (alice_qubit, bob_qubit) for pair i."""
        return (i, total_qubits - 1 - i)
    
    # Survivor is the innermost pair
    surv_A, surv_B = get_pair(num_rounds)
    
    # 1. Optional DEJMPS Twirling
    if use_dejmps:
        for q in range(total_qubits):
            qc.rx(np.pi/2, q)
    
    # 2. Bilateral CNOTs: Survivor controls -> Sacrificial targets
    for i in range(num_rounds):
        sac_A, sac_B = get_pair(i)
        qc.cx(surv_A, sac_A)  # Alice side
        qc.cx(surv_B, sac_B)  # Bob side
    
    # 3. Optional DEJMPS Untwirling (survivor only)
    if use_dejmps:
        qc.rx(-np.pi/2, surv_A)
        qc.rx(-np.pi/2, surv_B)
    
    # 4. Measure sacrificial pairs with feedforward
    for i in range(num_rounds):
        sac_A, sac_B = get_pair(i)
        
        qc.measure(sac_A, mA[i])
        
        with qc.if_test((mA[i], 1)):
            qc.x(sac_B)
        
        qc.measure(sac_B, parity[i])
    
    # 5. Compute global flag = OR(all parity bits)
    scratch = 0
    qc.reset(scratch)
    
    max_val = 1 << num_rounds
    for val in range(1, max_val):
        with qc.if_test((parity, val)):
            qc.x(scratch)
    
    qc.measure(scratch, flag[0])
    
    return qc

# Quick test
print("BBPSSW (3 rounds):")
print(create_distillation_circuit(num_rounds=3, use_dejmps=False).draw(output='text'))

## Step 5: Claim an Edge

In [ ]:
# Find claimable edges
claimable = client.get_claimable_edges()
claimable_sorted = sorted(claimable, key=lambda e: (e['difficulty_rating'], e['base_threshold']))

print(f"Claimable edges ({len(claimable)}):")
for edge in claimable_sorted[:5]:
    print(f"  {edge['edge_id']} - threshold: {edge['base_threshold']:.3f}, difficulty: {edge['difficulty_rating']}")

In [ ]:
# --- IMPROVED AUTONOMOUS TERRITORY BOT ---

def auto_occupy_territory():
    """
    Smart bot that:
    - Tries multiple edges, not just one
    - Retries failed attempts (they're FREE!)
    - Adapts configuration based on edge difficulty
    - Manages budget carefully
    """
    import time
    
    # Configuration
    MAX_RETRIES_PER_CONFIG = 5   # Retry same config multiple times (free!)
    MIN_BUDGET = 5
    
    # Define circuit configs to try: (num_rounds, use_dejmps, cost)
    CONFIGS = [
        (3, False, 4),  # BBPSSW 3 rounds - highest fidelity
        (3, True, 4),   # DEJMPS 3 rounds - alternate
        (2, False, 3),  # BBPSSW 2 rounds - medium
        (1, False, 2),  # BBPSSW 1 round - cheapest
    ]
    
    edges_tried = set()  # Track edges we've exhausted attempts on
    
    while True:
        status = client.get_status()
        budget = status.get('budget', 0)
        print(f"\n{'='*50}")
        print(f"Budget: {budget}")
        
        if budget < MIN_BUDGET:
            print("Budget too low. Stopping.")
            break
        
        # Get claimable edges
        claimable = client.get_claimable_edges()
        if not claimable:
            print("No claimable edges!")
            break
        
        # Sort by difficulty and threshold (easiest first)
        claimable.sort(key=lambda e: (e['difficulty_rating'], e['base_threshold']))
        
        # Filter out edges we've already exhausted
        available = [e for e in claimable if tuple(e['edge_id']) not in edges_tried]
        
        if not available:
            print("Tried all available edges. Resetting tried list...")
            edges_tried.clear()
            available = claimable
        
        target = available[0]
        edge_id = tuple(target['edge_id'])
        threshold = target['base_threshold']
        difficulty = target['difficulty_rating']
        
        print(f"Target: {edge_id}")
        print(f"  Difficulty: {difficulty}, Threshold: {threshold:.3f}")
        
        claimed = False
        
        # Try each configuration
        for num_rounds, use_dejmps, cost in CONFIGS:
            if budget < cost:
                continue
            
            variant = "DEJMPS" if use_dejmps else "BBPSSW"
            print(f"\n  Config: {variant} r={num_rounds} (cost={cost})")
            
            circuit = create_distillation_circuit(num_rounds=num_rounds, use_dejmps=use_dejmps)
            flag_bit = 2 * num_rounds
            
            # Retry same config multiple times (failed attempts are FREE)
            for attempt in range(1, MAX_RETRIES_PER_CONFIG + 1):
                try:
                    result = client.claim_edge(edge_id, circuit, flag_bit, cost)
                    
                    if result.get("ok"):
                        data = result["data"]
                        success = data.get("success", False)
                        fidelity = data.get("fidelity", 0)
                        prob = data.get("success_probability", 0)
                        
                        status_str = "✓ SUCCESS" if success else "✗ fail"
                        print(f"    [{attempt}] {status_str} F={fidelity:.3f} P={prob:.3f}")
                        
                        if success:
                            print(f"  >>> CLAIMED! <<<")
                            claimed = True
                            break
                    else:
                        err = result.get('error', {})
                        print(f"    [{attempt}] API Error: {err.get('message', err)}")
                        break
                        
                except Exception as e:
                    print(f"    [{attempt}] Error: {e}")
                    break
                
                time.sleep(0.5)
            
            if claimed:
                break
        
        if not claimed:
            print(f"  Could not claim this edge. Moving to next...")
            edges_tried.add(edge_id)
        
        time.sleep(1)
    
    print("\n" + "="*50)
    print("Bot finished!")
    client.print_status()

# RUN
auto_occupy_territory()

## Step 6: Check Progress

In [ ]:
client.print_status()

In [ ]:
# View leaderboard
leaderboard = client.get_leaderboard()["leaderboard"]
print("Leaderboard:")
for i, p in enumerate(leaderboard[:10]):
    print(f"{i+1}. {p.get('player_id', 'Unknown'):20} Score: {p.get('score', 0)}")

## Tips

- **Failed attempts are free** - only successful claims cost bell pairs
- **More bell pairs** can improve fidelity but cost more budget
- **Vertex rewards** are competitive - top players by claim strength earn rewards
- **Budget management** is key - if budget reaches 0, you're eliminated

Good luck!

In [ ]:
# Restart game (uncomment to use)
# result = client.restart()
# print(result)